In [1]:
# Time each cell
%load_ext autotime

In [2]:
import os

from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd
from glob import glob

from tqdm import tqdm_notebook

import numpy as np
from astropy.io import fits
from astropy.nddata.utils import Cutout2D, PartialOverlapError, NoOverlapError
from astropy.stats import sigma_clipped_stats
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy.time import Time

from photutils import find_peaks

from dateutil.parser import parse as date_parse

from scipy import linalg
from scipy import optimize

from copy import copy

from piaa.utils import helpers
from piaa.utils import images as img_utils
from piaa import exoplanets
from piaa.observation import Observation
from pocs.utils.images import fits as fits_utils

palette = copy(plt.cm.inferno)
palette.set_over('w', 1.0)
palette.set_under('k', 1.0)
palette.set_bad('g', 1.0)

/home/wtylergee/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


time: 10.3 s


/home/wtylergee/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


---
### 2. Pick a target

Here we manually looked up the 2MASS id for the Wasp-43, the target we are interested in.

In [3]:
os.environ['PGPASSWORD'] = 'pan0pt3&-t3ss'

time: 1.73 ms


In [4]:

# Get the PICID for the target via a manual lookup of the 2MASS id

# ticid=tic_list[np.random.randint(0, len(tic_list))] # random
# ticid = str(get_star_info('08154797+0550121')[0][0]) # 2MASS id - Hat-P-30
# ticid = str(get_star_info('07273995+2420118')[0][0]) # 2MASS id - Hat-P-20
# ticid = str(helpers.get_star_info('10193800-0948225')[0][0]) # 2MASS id - Wasp-43
# ticid = str(get_star_info('02265106+3733017')[0][0]) # 2MASS id - Wasp-33
# ticid = str(get_star_info('20004297+2242342')[0][0]) # HD 189733

# ticid = str(helpers.get_star_info('10422458+0726060')[0][0]) # 2MASS id - Wasp 104

ticid = str(helpers.get_star_info('17520702+3732461')[0][0]) # 2MASS id - Tres 3

# ticid = str(helpers.get_star_info('19272912+4405145')[0][0]) # 2MASS id - Kepler 1100
# 
# ticid = str(helpers.get_star_info('09543439+4023170')[0][0]) # 2MASS id - Kelt-3

# ticid = str(helpers.get_star_info('17430345+3734108')[0][0]) # 2MASS id - V966 Her δScuti

# ticid = str(helpers.get_star_info('18155058+3435358')[0][0]) # 2MASS id - HIP 89501

# ticid = str(helpers.get_star_info('18083550+3312028')[0][0]) # 2MASS id - HIP 88867

# ticid = str(helpers.get_star_info('20004370+2242391')[0][0]) # 2MASS id - HD 189733

# ticid = str(helpers.get_star_info('20124017-0208391')[0][0]) # 2MASS id - Wasp 80

# ticid = str(helpers.get_star_info('15085174+0220358', table='ctl')[0][0]) # 2MASS id - Wasp 24

ticid

'116264089'

time: 346 ms


In [5]:
rm /var/panoptes/images/lc/*_243921117_diff.csv

time: 129 ms


#### Run reduction

Here we have a fixed `num_refs` and damping value `d` and make a `run_piaa` method that runs reduction steps.

In [6]:
def do_plot(observation, lc, transit_info):
    plt.figure(figsize=(19,9))

    ingress_phase = observation.planet.get_phase(transit_info.ingress)
    egress_phase = observation.planet.get_phase(transit_info.egress)

    obs_start_phase = observation.planet.get_phase(observation.image_times[0])
    obs_stop_phase = observation.planet.get_phase(observation.image_times[-1])

    # phase = np.arange(len(target_psc))
    phase = [observation.planet.get_phase(Time(t0, format='mjd')) for t0 in lc.index]

#     d0 = lc - lc.mean()
    d0 = lc
    
    for color in 'rgb':
        # Offset lines from each other
    #     offset = i / 20
    #     d0 += offset
        offset = 0

        plt.plot(phase, d0['{}_target'.format(color)], color=color, label='{}_target'.format(color), marker='o')    
        plt.plot(phase, d0['{}_ideal'.format(color)], color=color, ls='--', label='{}_ideal'.format(color), marker='o')

        plt.hlines(1 + offset, phase[0], phase[-1], linestyles='dashed', alpha=0.5)
        plt.hlines(1.01 + offset, phase[0], phase[-1], linestyles='dashed', alpha=0.5, color='c')
        plt.hlines(.99 + offset, phase[0], phase[-1], linestyles='dashed', alpha=0.5, color='c')    

    plt.axvline(ingress_phase, 0, color='k', ls='--', alpha=0.5, label='ingress')
    plt.axvline(0, 0, color='k', ls='-.', alpha=0.25, label='midpoint')
    plt.axvline(egress_phase, 0, color='k', ls='--', alpha=0.3, label='egress')

    # plt.ylim(0.9, 1.1)
    # plt.xlim(0, len(target_psc))
    plt.xlim(phase[0] - .002, phase[-1] + .002)

    plt.legend()
    plt.title("{} - {} - {} - V={} - {:.02%}".format(
        observation.unit_id,
        observation.seq_time,
        observation.field,
        observation.planet.star_mag, 
        observation.planet.transit_depth / 100
    ))
    plt.tight_layout()
    plt.savefig('/var/panoptes/images/lc/{}.png'.format(observation.sequence.replace('/','_')))

time: 8.87 ms


In [11]:
with open('/var/panoptes/wasp43_seq.txt') as f:
    sequences = f.read().split('\n')
#     sequences.pop() # Remove final newline

time: 1.73 ms


In [12]:
sequences.reverse()
print(len(sequences))

4
time: 1.17 ms


In [13]:
sequences = [s for s in sequences if '14d3bd' in s]

time: 1.2 ms


In [14]:
sequences

['PAN001/Wasp43/14d3bd/20180307T081001',
 'PAN001/Wasp43/14d3bd/20180131T085025']

time: 3.65 ms


In [15]:
num_refs = 99

aperture_size = 6

picid = ticid

# for seq in sequences:

stamps = list()
stamp_size = (14, 14)

while sequences:
    seq = sequences.pop() # Don't do this
        
    if seq == '': continue
    print()
    print("Starting", seq)
    
    observation = Observation(seq, verbose=True)
    
    try:
        observation.get_stamps(
            upload=False,
            force_new=True,
            stamp_size=stamp_size,
        )
    except Exception as e:
        print(e)
        continue
            
    try:
        if ticid not in observation.stamps:
            print(ticid, "not in observation, skipping")
#             del observation
            continue
    except Exception as e:
        print(e)
        continue

    try:
        target_psc, ideal = observation.run_piaa(
            picid, 
            num_refs=num_refs, 
            aperture_size=aperture_size, 
            force_new=True, 
            verbose=True, 
            show_stamps=True,
            subtract_bias=True
        )
    except Exception as e:
        print("Problem: {}".format(e))
        continue
        
    stamps.append((target_psc, ideal))
        
    try:
        fn = '/var/panoptes/images/lc/{}_{}_diff.csv'.format(observation.sequence.replace('/', '_'), picid)
        lc = pd.read_csv(fn, parse_dates=True, index_col=[0])
    except FileNotFoundError:
        print("Can't find diff file, skipping")
        continue
        
    try:
        for t0 in observation.image_times:
            in_transit, transit_info = observation.planet.in_transit(t0, with_times=True)
            if in_transit:
                print("Making plot")
                do_plot(observation, lc, transit_info)
                break
    except Exception as e:
        print(e)
                
#     try:
#         frame_idx = 0
#         aperture_position = (lc.aperture_x[frame_idx], lc.aperture_y[frame_idx])
#         helpers.show_stamps(
#             pscs=[
#                 target_psc.reshape(observation.num_frames, stamp_size[0], stamp_size[1]), 
#                 ideal.reshape(observation.num_frames, stamp_size[0], stamp_size[1])
#             ], 
#             stamp_size=stamp_size[0], 
#             aperture_size=aperture_size,
#             aperture_position=aperture_position,
#             frame_idx=frame_idx) #, stretch='log')           
#     except Exception as e:
#         print(e)
    
    print("Done with", seq)
    print()
    
#     del observation


Starting PAN001/Wasp43/14d3bd/20180131T085025


Getting point sources    :   0%|          | 0/60 [00:00<?, ?it/s]


'DataFrame' object has no attribute 'snr'

Starting PAN001/Wasp43/14d3bd/20180307T081001


KeyboardInterrupt: 

time: 1min 33s


In [13]:
observation.stamps

<HDF5 group "/stamps" (9382 members)>

time: 15 ms


---
**Scratchpad below**